In [1]:
# importing important libraries for need in work for bulding CNN model
import os
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [6]:
# Check contents of subdirectories and which directory have images
subdirs = ["UTKFace", "utkface_aligned_cropped", "crop_part1"]

for subdir in subdirs:
    full_path = os.path.join(data_dir, subdir)
    print(f"Contents of {subdir}:")
    print(os.listdir(full_path)[:10])  # Print first 10 files in each subdirectory
# we can see utk face and crop part 1 contain images
# so lets proceed with utk face

Contents of UTKFace:
['26_0_2_20170104023102422.jpg.chip.jpg', '22_1_1_20170112233644761.jpg.chip.jpg', '21_1_3_20170105003215901.jpg.chip.jpg', '28_0_0_20170117180555824.jpg.chip.jpg', '17_1_4_20170103222931966.jpg.chip.jpg', '44_0_3_20170119201022260.jpg.chip.jpg', '35_0_2_20170116182734834.jpg.chip.jpg', '76_0_0_20170104213515132.jpg.chip.jpg', '36_1_0_20170116165722892.jpg.chip.jpg', '34_0_3_20170119200815948.jpg.chip.jpg']
Contents of utkface_aligned_cropped:
['UTKFace', 'crop_part1']
Contents of crop_part1:
['26_0_2_20170104023102422.jpg.chip.jpg', '21_1_3_20170105003215901.jpg.chip.jpg', '17_1_4_20170103222931966.jpg.chip.jpg', '76_0_0_20170104213515132.jpg.chip.jpg', '18_1_0_20170104022856102.jpg.chip.jpg', '12_1_0_20170109203700059.jpg.chip.jpg', '4_0_0_20170110212940437.jpg.chip.jpg', '67_0_0_20170111193821219.jpg.chip.jpg', '26_1_2_20170104021513981.jpg.chip.jpg', '90_0_2_20170111210740854.jpg.chip.jpg']


In [7]:
# Set the directory to the 'UTKFace' subfolder
data_dir = "/kaggle/input/utkface-new/UTKFace"

In [10]:
images = []
ages = []

for img_name in os.listdir(data_dir):
    if img_name.endswith(".jpg") or img_name.endswith(".png"):  # Include other extensions if necessary
        try:
            # Extract age from the filename (assuming age is the first part of the filename)
            age = int(img_name.split('_')[0])
            img_path = os.path.join(data_dir, img_name)
            
            # Read and process the image
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (64, 64))  # Resize to 64x64
                images.append(img)
                ages.append(age)
            else:
                print(f"Could not read image: {img_name}")
        except Exception as e:
            print(f"Error processing file {img_name}: {e}")


In [11]:
# Check the number of images and ages loaded
print(f"Number of images loaded: {len(images)}")
print(f"Number of ages loaded: {len(ages)}")


Number of images loaded: 23708
Number of ages loaded: 23708


In [12]:
from sklearn.model_selection import train_test_split

# Convert the lists to numpy arrays for training
import numpy as np
X = np.array(images)
y = np.array(ages)

In [13]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape}, Test set size: {X_test.shape}")

Training set size: (18966, 64, 64, 3), Test set size: (4742, 64, 64, 3)


In [14]:
# Sample CNN Model for Age Detection
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='linear')  # Output for age prediction
])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# training  the model with train dataset
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)
# evaluating model performance using (mae- mean absoulute error)

Epoch 1/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 55s 90ms/step - loss: 6324.8027 - mae: 23.6897 - val_loss: 205.6154 - val_mae: 10.2181
Epoch 2/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 53s 90ms/step - loss: 142.2805 - mae: 8.8914 - val_loss: 139.3751 - val_mae: 8.5795
Epoch 3/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 53s 89ms/step - loss: 116.4540 - mae: 8.0255 - val_loss: 142.4325 - val_mae: 8.4744
Epoch 4/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 54s 91ms/step - loss: 105.1056 - mae: 7.7186 - val_loss: 125.2507 - val_mae: 8.0681
Epoch 5/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 54s 91ms/step - loss: 93.4321 - mae: 7.2438 - val_loss: 126.0436 - val_mae: 8.1421
Epoch 6/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 81s 91ms/step - loss: 88.3062 - mae: 7.0095 - val_loss: 126.4752 - val_mae: 8.0707
Epoch 7/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 83s 92ms/step - loss: 80.0012 - mae: 6.7157 - val_loss: 128.3187 - val_mae: 8.1395
Epoch 8/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 81s 91ms/step - loss: 76.0674 - mae: 6.5501 - val_loss: 120.1900 - val_mae: 7.9050
Epoch 9/1

In [16]:
#now we are going to Save this model with h.5 extension
model.save("age_detection_cnn.h5")

In [ ]:
# now we are going to do task2
# the task was age detection through voice
# force the voice detection want to go with librosa